In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2525/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2_sup")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
179,model_9_9_15,0.999949,0.993371,0.999800,0.999779,0.999792,11.292484,1478.634821,9.487422e+00,15.327835,12.407628,0.046130,3.360429,1.000405,3.503490,49.151725,82.061373,"Hidden Size=[2, 5], regularizer=0.2, learning_..."
181,model_9_9_14,0.999948,0.993472,0.999820,0.999840,0.999836,11.587261,1456.082762,8.543627e+00,11.087526,9.815576,0.047110,3.404007,1.000416,3.548922,49.100187,82.009835,"Hidden Size=[2, 5], regularizer=0.2, learning_..."
182,model_9_9_13,0.999948,0.993474,0.999817,0.999835,0.999832,11.696061,1455.621482,8.650071e+00,11.413807,10.031939,0.047252,3.419950,1.000419,3.565545,49.081496,81.991143,"Hidden Size=[2, 5], regularizer=0.2, learning_..."
183,model_9_9_12,0.999947,0.993491,0.999821,0.999836,0.999834,11.857719,1451.776685,8.483443e+00,11.334247,9.908845,0.047257,3.443504,1.000425,3.590101,49.054042,81.963689,"Hidden Size=[2, 5], regularizer=0.2, learning_..."
189,model_9_9_11,0.999942,0.993552,0.999824,0.999857,0.999847,12.884133,1438.172110,8.318835e+00,9.919186,9.119011,0.047035,3.589447,1.000462,3.742258,48.888007,81.797654,"Hidden Size=[2, 5], regularizer=0.2, learning_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,model_34_6_21,0.996589,0.996050,1.000000,0.999858,0.999966,760.873282,881.070168,6.227254e-07,10.765588,5.382794,0.014725,27.583932,1.005458,28.758237,64.731066,112.267224,"Hidden Size=[2, 8], regularizer=0.2, learning_..."
1977,model_34_6_2,0.996589,0.996050,1.000000,0.999858,0.999966,760.873282,881.070168,6.227254e-07,10.765588,5.382794,0.014725,27.583932,1.005458,28.758237,64.731066,112.267224,"Hidden Size=[2, 8], regularizer=0.2, learning_..."
1988,model_34_6_0,0.996589,0.996050,1.000000,0.999858,0.999966,760.873282,881.070168,6.227254e-07,10.765588,5.382794,0.014725,27.583932,1.005458,28.758237,64.731066,112.267224,"Hidden Size=[2, 8], regularizer=0.2, learning_..."
1989,model_34_6_22,0.996589,0.996050,1.000000,0.999858,0.999966,760.873282,881.070168,6.227254e-07,10.765588,5.382794,0.014725,27.583932,1.005458,28.758237,64.731066,112.267224,"Hidden Size=[2, 8], regularizer=0.2, learning_..."
